In [5]:
import sys 
import numpy as np
import torch
import torchvision

sys.path.append('../firedetect')
from model import load_dataset

In [6]:
model = torch.load('../weights/resnet50-epoch-5-valid_acc=0.9906-test_acc=0.6241.pt')

dataset_paths = {'mine': '/home/013855803/fire_aerial2k_dataset/',
                 'dunnings': '/home/013855803/fire-dataset-dunnings/images-224x224/train',
                 'dunnings_test': '/home/013855803/fire-dataset-dunnings/images-224x224/test'}

In [7]:
def accuracy_gpu(pred, truth):
    agreeing = pred.eq(truth)
    acc = float(agreeing.sum())/agreeing.numel()
    return acc

In [12]:
import torch
import torchvision
import sklearn.metrics


tr = torchvision.transforms.Compose([torchvision.transforms.Resize((224,224)),
                            torchvision.transforms.ToTensor()])

test_dataset = torchvision.datasets.ImageFolder(root=dataset_paths['dunnings_test'],
                                                transform=tr)

# test_dataset.class_to_idx = {'fire': 1, 'nofire': 0} # for dunnings

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=64,
    num_workers=0,
    shuffle=True
)

device = torch.device("cuda:0")
test_acc = []

model = model.to(device)

for param in model.parameters():
    param.requires_grad = False

for i, data in enumerate(test_loader):
    print(f'testing batch {i}/{len(test_loader)}')
    inputs = data[0].to(device)
    labels = torch.tensor(data[1], dtype=torch.bool).to(device)

    scores = model(inputs)
    pred = scores.squeeze() > 0.5
    
    a = accuracy_gpu(pred, labels)
    test_acc.append(a)
    print(np.mean(test_acc))

testing batch 0/46
0.953125


/home/013855803/anaconda3/envs/fire-detect/lib/python3.7/site-packages/ipykernel_launcher.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


testing batch 1/46
0.921875
testing batch 2/46
0.9427083333333334
testing batch 3/46
0.94140625
testing batch 4/46
0.95
testing batch 5/46
0.9505208333333334
testing batch 6/46
0.9486607142857143
testing batch 7/46
0.9453125
testing batch 8/46
0.9444444444444444
testing batch 9/46
0.946875
testing batch 10/46
0.9502840909090909
testing batch 11/46
0.9388020833333334
testing batch 12/46
0.9399038461538461
testing batch 13/46
0.9341517857142857
testing batch 14/46
0.9354166666666667
testing batch 15/46
0.9326171875
testing batch 16/46
0.9283088235294118
testing batch 17/46
0.9296875
testing batch 18/46
0.928453947368421
testing batch 19/46
0.92890625
testing batch 20/46
0.9285714285714286
testing batch 21/46
0.9318181818181818
testing batch 22/46
0.9334239130434783
testing batch 23/46
0.9348958333333334
testing batch 24/46
0.93625
testing batch 25/46
0.9375
testing batch 26/46
0.9363425925925926
testing batch 27/46
0.9375
testing batch 28/46
0.9380387931034483
testing batch 29/46
0.93697

In [ ]:
from model import Model, load_dataset, accuracy
import numpy as np
import torch
import torchvision

BACKBONES = ['resnet18','resnet34','resnet50','resnet101', 'densenet121', 'mobilenet']
BACKBONES = ['resnet50'] # override with just one backbone
BATCH_SIZE = 16

dataset_paths = {'mine': '/home/013855803/fire_aerial2k_dataset/',
                 'dunnings': '/home/013855803/fire-dataset-dunnings/images-224x224/train',
                 'dunnings_test': '/home/013855803/fire-dataset-dunnings/images-224x224/test'}


train_loader, valid_loader = load_dataset(dataset_paths['dunnings'], batch_size=BATCH_SIZE)

tr = torchvision.transforms.Compose([torchvision.transforms.Resize((224,224)),
                            torchvision.transforms.ToTensor()])

test_dataset = torchvision.datasets.ImageFolder(root=dataset_paths['dunnings_test'],
                                                transform=tr)


test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=4,
    shuffle=False
)

print(f'Loaded {len(train_loader)} training batches and {len(train_loader) * BATCH_SIZE} samples')
print(f'Loaded {len(valid_loader)} training batches and {len(valid_loader) * BATCH_SIZE} samples')
print(f'Loaded {len(test_loader)} training batches and {len(test_loader) * BATCH_SIZE} samples')

# Can be useful if we're retraining many times on the entire dataset
# completely memory extravagant but I have 256GB of RAM to use :)
# train, valid = list(train), list(valid)

In [ ]:
import torch
device = torch.device("cuda:0")
is_validating = True
is_testing = True

history = {
    'train_samples': [],
    'train_acc': [],
    'valid_acc': [],
    'test_acc': [],
    'loss': []
}


for b in BACKBONES:

    import torch.optim as optim

    m = Model(backbone=b)
    m = m.to(device)

    criterion = torch.nn.BCELoss()

    for epoch in range(10): # epochs

        optimizer = optim.Adam(m.parameters(), lr= 1e-4 if epoch <5 else 1e-5, weight_decay=1e-3)
        
        running_loss = []
        running_acc = []

        # epoch training
        for i, data in enumerate(train_loader):

            # get the inputs; data is a list of [inputs, labels]
            inputs = data[0].to(device)
            labels = data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = m(inputs)
            loss = criterion(outputs[:,0], labels.type_as(outputs[:,0]))
            loss.backward()
            optimizer.step()

            acc = accuracy(outputs, labels)
            # print statistics

            running_loss.append(loss.item())
            running_acc.append(acc)


            if i%20 == 0:
                print(f'epoch: {epoch+1:02d}, batch: {i:03d}, \
                loss: {np.mean(running_loss):.3f}, training accuracy: {np.mean(running_acc):.3f}')

                history['loss'].append(np.mean(running_loss))
                history['train_samples'].append(epoch*len(train_loader)+i)
                history['train_acc'].append(np.mean(running_acc))
                
                
        # on epoch end:
        if is_validating:
            valid_acc = []
            # epoch validation
            for i, data in enumerate(valid_loader):
                # get the inputs; data is a list of [inputs, labels]
                inputs = data[0].to(device)
                labels = data[1].to(device)

                # could pehaps do:
                # for param in m.parameters():
                #     param.requires_grad = False

                outputs = m(inputs)
                valid_acc.append(accuracy(outputs, labels))
            va = round(np.mean(valid_acc), 4)
            print(f'validation accuracy {va}')
            history['valid_acc'].append(va)
        else:
            va='-1'

        if is_testing:
            test_acc = []
            # epoch validation
            for i, data in enumerate(test_loader):
                # get the inputs; data is a list of [inputs, labels]
                inputs = data[0].to(device)
                labels = data[1].to(device)

                # could pehaps do:
                # for param in m.parameters():
                #     param.requires_grad = False

                outputs = m(inputs)
                test_acc.append(accuracy(outputs, labels))
            tst = round(np.mean(test_acc), 4)
            print(f'test_accuracy {tst}')
            history['test_acc'].append(tst)
        else:
            tst = '-1'

        fname =  f'weights/{b}-epoch-{epoch}-valid_acc={va}-test_acc={tst}.pt'
        print(f'Saved {fname}')
        torch.save(m, fname)


    print(f'Finished Training: {b}')

In [ ]:
import matplotlib.pyplot as plt

# for history in histories:
plt.figure()
plt.plot(history['train_samples'], history['loss'])

In [ ]:
plt.plot(history['train_samples'], history['train_acc'])

In [ ]:
history['valid_acc']

In [ ]:
plt.scatter(np.array(history['train_samples'])/len(train_loader), history['train_acc'])
plt.scatter(list(range(10)), history['valid_acc'])
plt.scatter(list(range(10)), history['test_acc'])

In [ ]:
for k,v in histories.items():
    print(v['train_acc'])

In [ ]:
history.keys()

